In [1]:
from datasets import load_dataset

emotion_dataset = load_dataset("emotion")
emotion_dataset

/home/saxenaya/miniconda3/envs/robot_commands/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: emotion/split
Found cached dataset emotion (/home/saxenaya/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)
100%|██████████| 3/3 [00:00<00:00, 402.85it/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [2]:
emotion_dataset["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [3]:
emotion_df = emotion_dataset["train"].to_pandas()
emotion_df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [4]:
features = emotion_dataset['train'].features
features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [5]:
features['label'].int2str(0)

'sadness'

In [6]:
id2label = {idx:features['label'].int2str(idx) for idx in range(6)}
id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [7]:
label2id = {v:k for k,v in id2label.items()}
label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [8]:
emotion_df['label'].value_counts(normalize=True).sort_index()

0    0.291625
1    0.335125
2    0.081500
3    0.134937
4    0.121063
5    0.035750
Name: label, dtype: float64

In [9]:
from transformers import AutoTokenizer

model_ckpt = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [10]:
tokenizer(emotion_dataset["train"]["text"][:1])

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [11]:
def tokenize_text(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512)

In [12]:
emotion_dataset = emotion_dataset.map(tokenize_text, batched=True)
emotion_dataset

Loading cached processed dataset at /home/saxenaya/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd/cache-86b0e6c4497211f9.arrow
Loading cached processed dataset at /home/saxenaya/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd/cache-7e4d0276b440a070.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [13]:
class_weights = (1 - (emotion_df['label'].value_counts().sort_index() / len(emotion_df))).values
class_weights

array([0.708375 , 0.664875 , 0.9185   , 0.8650625, 0.8789375, 0.96425  ])

In [14]:
import torch

class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643], device='cuda:0')

In [15]:
# Need column name of target to be "labels"
emotion_dataset = emotion_dataset.rename_column("label", "labels")

In [16]:
from torch import nn
import torch
from transformers import Trainer
class WeightedLossTrainer(Trainer):
    def compute_loss (self, model, inputs, return_outputs=False):
        # Feed inputs to model and extract logits
        outputs = model(*inputs)
        logits = outputs.get("logits")
        # Extract labels
        labels = inputs.get("labels'")
        # Define loss function wi th class weights
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        # Compute loss
        loss= loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [17]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=6, id2label=id2label, label2id=label2id, return_dict=False
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from sklearn.metrics import f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    return f1

In [19]:
from transformers import TrainingArguments

batch_size = 64
#Log the training loss at each epoch
logging_steps = len(emotion_dataset ["train"]) // batch_size
output_dir ="minilm-finetuned-emotion"

training_args  = TrainingArguments(output_dir=output_dir,
                                    num_train_epochs=5,
                                    learning_rate=2e-5,
                                    per_device_train_batch_size=batch_size,
                                    per_device_eval_batch_size=batch_size,
                                    weight_decay=0.01,
                                    evaluation_strategy="epoch",
                                    logging_steps=logging_steps,
                                    fp16=True, # Make it train fast!
                                    )

In [20]:
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=emotion_dataset["train"],
    eval_dataset=emotion_dataset["validation"],
    tokenizer=tokenizer,
)

In [21]:
trainer.train()

/home/saxenaya/miniconda3/envs/robot_commands/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


AttributeError: Caught AttributeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/saxenaya/miniconda3/envs/robot_commands/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/saxenaya/miniconda3/envs/robot_commands/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/saxenaya/miniconda3/envs/robot_commands/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py", line 1562, in forward
    outputs = self.bert(
  File "/home/saxenaya/miniconda3/envs/robot_commands/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/saxenaya/miniconda3/envs/robot_commands/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py", line 968, in forward
    input_shape = input_ids.size()
